In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import json
import warnings
from typing import Dict, List, Optional, Tuple, Union
from functools import partial
from tempfile import gettempdir
import matplotlib.pyplot as plt
import time
from tqdm import tqdm
from prettytable import PrettyTable
from pathlib import Path
import os
import cv2

#-----------------l5kit lib ------------------------------------------
from l5kit.configs import load_config_data
from l5kit.data import (
    LocalDataManager, ChunkedDataset, DataManager,
    get_agents_slice_from_frames, get_frames_slice_from_scenes,get_tl_faces_slice_from_frames,
    filter_agents_by_labels, filter_tl_faces_by_frames
)
from l5kit.data.labels import PERCEPTION_LABEL_TO_INDEX 
from l5kit.data.filter import filter_agents_by_labels, filter_agents_by_track_id, filter_agents_by_frames
from l5kit.dataset import AgentDataset, EgoDataset
from l5kit.rasterization import (
    build_rasterizer, Rasterizer, RenderContext,
    EGO_EXTENT_HEIGHT, EGO_EXTENT_LENGTH, EGO_EXTENT_WIDTH
)
from l5kit.rasterization.sem_box_rasterizer import SemBoxRasterizer
from l5kit.rasterization.semantic_rasterizer import CV2_SHIFT, cv2_subpixel,SemanticRasterizer
from l5kit.evaluation import write_pred_csv, compute_metrics_csv, read_gt_csv, create_chopped_dataset
from l5kit.evaluation.chop_dataset import MIN_FUTURE_STEPS
from l5kit.evaluation.metrics import neg_multi_log_likelihood, time_displace
from l5kit.geometry import transform_points, angular_distance, compute_agent_pose, rotation33_as_yaw,transform_point
from l5kit.geometry.transform import yaw_as_rotation33
from l5kit.visualization import PREDICTED_POINTS_COLOR, TARGET_POINTS_COLOR, draw_trajectory
from l5kit.kinematic import Perturbation
from l5kit.sampling.slicing import get_future_slice, get_history_slice
from l5kit.data.zarr_dataset import AGENT_DTYPE

#------------ Tensorflow lib ---------------------------------------------
import tensorflow as tf
import tensorflow.keras as tfk
import tensorflow.keras.layers as tfkl
from tensorflow.data import Dataset

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))
#
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
cfg = {
    'format_version': 4,
    'model_params': {
        'history_num_frames': 10,
        'history_step_size': 1,
        'history_delta_time': 0.1,
        'future_num_frames': 50,
        'future_step_size': 1,
        'future_delta_time': 0.1
    },
    
    'raster_params': {
        'raster_size': [224, 224],
        'pixel_size': [0.5, 0.5],
        'ego_center': [0.25, 0.5],
        'map_type': 'py_semantic',
        'satellite_map_key': 'aerial_map/aerial_map.png',
        'semantic_map_key': 'semantic_map/semantic_map.pb',
        'dataset_meta_key': 'meta.json',
        'filter_agents_threshold': 0.5,
        'disable_traffic_light_faces': False
    },
    
    'train_data_loader': {
        'key': 'scenes/test.zarr',
        'weights' : "/kaggle/input/lift5k-weights-mobilenet/weight_mobile_net_19229.hdf5",
        'load_weights' : False,
        'input_deep' : 69,
        'batch_size': 12,
        'shuffle': True,
        'num_workers': 4
    }
}

DIR_INPUT = "/kaggle/input/lyft-motion-prediction-autonomous-vehicles"
MIN_FRAME_HISTORY = 10
MIN_FRAME_FUTURE = 1
os.environ["L5KIT_DATA_FOLDER"] = DIR_INPUT
dm = LocalDataManager(None)
raster_w = cfg["raster_params"]["raster_size"][0]
raster_h = cfg["raster_params"]["raster_size"][1]
input_deep = cfg["train_data_loader"]["input_deep"]
length_of_rnn_sequence = cfg["model_params"]["history_num_frames"] + 1
future_output_shape = (cfg["model_params"]["future_num_frames"], 2)
BATCHSIZE = cfg["train_data_loader"]["batch_size"]
train_zarr = ChunkedDataset(dm.require(cfg["train_data_loader"]["key"])).open()
weights_filepath = cfg["train_data_loader"]["weights"]
load_weights = cfg["train_data_loader"]["load_weights"]
mob_net = tfk.applications.MobileNet(include_top=False, pooling = "avg")

In [ ]:
def expand_into_deep(num_of_layers, raster_size: Tuple[int, int], raster_from_world: np.ndarray, agents: np.ndarray,
                     ego_index: int) -> np.ndarray:
    layers = np.zeros((num_of_layers,raster_size[1], raster_size[0]), dtype=np.float32)
    box_world_coords = np.zeros((len(agents), 4, 2))
    corners_base_coords = np.asarray([[-1, -1], [-1, 1], [1, 1], [1, -1]])

    for idx, agent in enumerate(agents):
        corners = corners_base_coords * agent["extent"][:2] / 2  # corners in zero
        r_m = yaw_as_rotation33(agent["yaw"])
        box_world_coords[idx] = transform_points(corners, r_m) + agent["centroid"][:2]
    
    box_raster_coords = transform_points(box_world_coords.reshape((-1, 2)), raster_from_world)
    box_raster_coords = cv2_subpixel(box_raster_coords.reshape((-1, 4, 2)))
    labels = ['PERCEPTION_LABEL_CAR', 'PERCEPTION_LABEL_BICYCLE', 'PERCEPTION_LABEL_PEDESTRIAN']
        
    for idx, agent in enumerate(agents):
        for i, label in enumerate(labels):
            lix = PERCEPTION_LABEL_TO_INDEX[label]
            color = 1.0 - float(agent["label_probabilities"][lix])
            cv2.fillPoly(layers[i,...], box_raster_coords[[idx]], color=color, shift=CV2_SHIFT)
        i = len(labels)
        cv2.fillPoly(layers[i,...], box_raster_coords[[idx]], color=float(agent["velocity"][0]), shift=CV2_SHIFT)
        cv2.fillPoly(layers[i + 1,...], box_raster_coords[[idx]], color=float(agent["velocity"][1]),shift=CV2_SHIFT) 
                
    if ego_index >= 0:
        cv2.fillPoly(layers[num_of_layers - 1,...], box_raster_coords[[ego_index]], color=1.0, shift=CV2_SHIFT) 
    return layers

def get_ego_as_agent(frame: np.ndarray) -> np.ndarray:  # TODO this can be useful to have around
    ego_agent = np.zeros(1, dtype=AGENT_DTYPE)
    ego_agent[0]["centroid"] = frame["ego_translation"][:2]
    ego_agent[0]["yaw"] = rotation33_as_yaw(frame["ego_rotation"])
    ego_agent[0]["extent"] = np.asarray((EGO_EXTENT_LENGTH, EGO_EXTENT_WIDTH, EGO_EXTENT_HEIGHT))
    return ego_agent

def _load_metadata(meta_key: str, data_manager: DataManager) -> dict:
    metadata_path = data_manager.require(meta_key)
    with open(metadata_path, "r") as f:
        metadata: dict = json.load(f)
    return metadata

class DecoratedSemBoxRasterizer(Rasterizer):
    def __init__(self,cfg: dict, data_manager: DataManager):
        super(DecoratedSemBoxRasterizer, self).__init__()
        raster_cfg = cfg["raster_params"]
        map_type = raster_cfg["map_type"]
        dataset_meta_key = raster_cfg["dataset_meta_key"]

        render_context = RenderContext(
            raster_size_px=np.array(raster_cfg["raster_size"]),
            pixel_size_m=np.array(raster_cfg["pixel_size"]),
            center_in_raster_ratio=np.array(raster_cfg["ego_center"]),)

        filter_agents_threshold = raster_cfg["filter_agents_threshold"]
        history_num_frames = cfg["model_params"]["history_num_frames"]
        semantic_map_path = data_manager.require(raster_cfg["semantic_map_key"])
        dataset_meta = _load_metadata(raster_cfg["dataset_meta_key"], data_manager)
        world_to_ecef = np.array(dataset_meta["world_to_ecef"], dtype=np.float64)
              
        self.render_context = render_context
        self.raster_size = render_context.raster_size_px
        self.pixel_size = render_context.pixel_size_m
        self.ego_center = render_context.center_in_raster_ratio
        self.filter_agents_threshold = filter_agents_threshold
        self.history_num_frames = history_num_frames
        self.sem_rast = SemanticRasterizer(render_context, semantic_map_path, world_to_ecef)

    def rasterize(self,history_frames: np.ndarray,history_agents: List[np.ndarray],
                  history_tl_faces: List[np.ndarray],agent: Optional[np.ndarray] = None,) -> np.ndarray:
        im_out_box = self.get_box_rasterization(history_frames,
                                                history_agents,
                                                history_tl_faces,
                                                agent ).reshape(self.raster_size[0], self.raster_size[1], -1)
        im_out_sem = self.sem_rast.rasterize(history_frames, history_agents, history_tl_faces, agent)
        return np.concatenate([im_out_box, im_out_sem], -1)
    
    def get_box_rasterization(self, history_frames: np.ndarray, history_agents: List[np.ndarray],history_tl_faces: List[np.ndarray],
                              agent: Optional[np.ndarray] = None):
        if agent is None:
            ego_translation_m = history_frames[0]["ego_translation"]
            ego_yaw_rad = rotation33_as_yaw(frame["ego_rotation"])
        else:
            ego_translation_m = np.append(agent["centroid"], history_frames[0]["ego_translation"][-1])
            ego_yaw_rad = agent["yaw"]
        num_of_layers = 6
        raster_from_world = self.render_context.raster_from_world(ego_translation_m, ego_yaw_rad)
        out_shape = (self.raster_size[1], self.raster_size[0], self.history_num_frames + 1, num_of_layers)
        agents_images = np.zeros(out_shape, dtype=np.float32)
        
        for i, (frame, agents) in enumerate(zip(history_frames, history_agents)):
            agents = filter_agents_by_labels(agents, self.filter_agents_threshold)
            av_index = len(agents)
            # note the cast is for legacy support of dataset before April 2020
            av_agent = get_ego_as_agent(frame).astype(agents.dtype)
            if agent is None:
                agents_image = expand_into_deep(num_of_layers, self.raster_size, raster_from_world,
                                                np.append(agents, av_agent), ego_index = -1)
            else:
                agent_ego = filter_agents_by_track_id(agents, agent["track_id"])
                if len(agent_ego) == 0:  # agent not in this history frame
                    agents_image = expand_into_deep(num_of_layers,self.raster_size, raster_from_world, np.append(agents, av_agent),
                                                    ego_index = -1)
                else:  # add av to agents and remove the agent from agents
                    #agents = agents[agents != agent_ego[0]]
                    ego_index = np.nonzero(agents == agent_ego[0])[0][0]
                    agents_image = expand_into_deep(num_of_layers,self.raster_size, raster_from_world, np.append(agents, av_agent), ego_index)
            agents_images[..., i,:num_of_layers] = agents_image.transpose((1,2,0))
        return agents_images
    
    def to_rgb(self, in_im: np.ndarray, **kwargs: dict) -> np.ndarray:
        pass
    
def _create_targets_for_deep_prediction(num_frames: int, frames: np.ndarray, selected_track_id: Optional[int], 
                                        agents: List[np.ndarray], agent_from_world: np.ndarray,current_agent_yaw: float
                                       ) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    coords_offset = np.zeros((num_frames, 2), dtype=np.float32)
    yaws_offset = np.zeros((num_frames, 1), dtype=np.float32)
    velocity_offset = np.zeros((num_frames, 2), dtype=np.float32)
    availability = np.zeros((num_frames,), dtype=np.float32)

    for i, (frame, frame_agents) in enumerate(zip(frames, agents)):
        if selected_track_id is None:
            agent_centroid = frame["ego_translation"][:2]
            agent_velocity = np.zeros(2,dtype = np.float32)
            agent_yaw = rotation33_as_yaw(frame["ego_rotation"])
        else:
            # it's not guaranteed the target will be in every frame
            try:
                agent = filter_agents_by_track_id(frame_agents, selected_track_id)[0]
                agent_centroid = agent["centroid"]
                agent_velocity = agent["velocity"]
                agent_yaw = agent["yaw"]
            except IndexError:
                availability[i] = 0.0  # keep track of invalid futures/history
                continue

        coords_offset[i] = transform_point(agent_centroid, agent_from_world)
        yaws_offset[i] = angular_distance(agent_yaw, current_agent_yaw)
        velocity_offset[i] = agent_velocity
        availability[i] = 1.0
    return coords_offset, yaws_offset, velocity_offset 

def decorated_generate_agent_sample(state_index: int, frames: np.ndarray, agents: np.ndarray, tl_faces: np.ndarray,
                                    selected_track_id: Optional[int], render_context: RenderContext, history_num_frames: int,
                                    history_step_size: int, future_num_frames: int, future_step_size: int,
                                    filter_agents_threshold: float, rasterizer: Rasterizer) -> dict:
    #  the history slice is ordered starting from the latest frame and goes backward in time., ex. slice(100, 91, -2)
    history_slice = get_history_slice(state_index, history_num_frames, history_step_size, include_current_state=True)
    future_slice = get_future_slice(state_index, future_num_frames, future_step_size)
    history_frames = frames[history_slice].copy()  # copy() required if the object is a np.ndarray
    future_frames = frames[future_slice].copy()
    sorted_frames = np.concatenate((history_frames[::-1], future_frames))  # from past to future
    agent_slice = get_agents_slice_from_frames(sorted_frames[0], sorted_frames[-1])
    agents = agents[agent_slice].copy()  # this is the minimum slice of agents we need
    history_frames["agent_index_interval"] -= agent_slice.start  # sync interval with the agents array
    future_frames["agent_index_interval"] -= agent_slice.start  # sync interval with the agents array
    history_agents = filter_agents_by_frames(history_frames, agents)
    future_agents = filter_agents_by_frames(future_frames, agents)
    tl_slice = get_tl_faces_slice_from_frames(history_frames[-1], history_frames[0])  # -1 is the farthest
    # sync interval with the traffic light faces array
    history_frames["traffic_light_faces_index_interval"] -= tl_slice.start
    history_tl_faces = filter_tl_faces_by_frames(history_frames, tl_faces[tl_slice].copy())
    # State you want to predict the future of.
    cur_frame = history_frames[0]
    cur_agents = history_agents[0]

    if selected_track_id is None:
        agent_centroid_m = cur_frame["ego_translation"][:2]
        agent_yaw_rad = rotation33_as_yaw(cur_frame["ego_rotation"])
        agent_extent_m = np.asarray((EGO_EXTENT_LENGTH, EGO_EXTENT_WIDTH, EGO_EXTENT_HEIGHT))
        agent_velocity_m = np.zero(2, dtype=np.float32)
        selected_agent = None
    else:
        try:
            agent = filter_agents_by_track_id(
                filter_agents_by_labels(cur_agents, filter_agents_threshold), selected_track_id
            )[0]
        except IndexError:
            raise ValueError(f" track_id {selected_track_id} not in frame or below threshold")
        agent_centroid_m = agent["centroid"]
        agent_yaw_rad = float(agent["yaw"])
        agent_extent_m = agent["extent"]
        agent_velocity_m = agent["velocity"]
        selected_agent = agent

    input_im = (
        None
        if not rasterizer
        else rasterizer.rasterize(history_frames, history_agents, history_tl_faces, selected_agent)
    )

    world_from_agent = compute_agent_pose(agent_centroid_m, agent_yaw_rad)
    agent_from_world = np.linalg.inv(world_from_agent)
    raster_from_world = render_context.raster_from_world(agent_centroid_m, agent_yaw_rad)

    future_coords_offset, future_yaws_offset, future_velocity_offset = _create_targets_for_deep_prediction(
        future_num_frames, future_frames, selected_track_id, future_agents, agent_from_world, agent_yaw_rad
    )

    return {
        "image": input_im,
        "target_positions": future_coords_offset,
        "target_velocity" : future_velocity_offset,
        "target_yaws": future_yaws_offset,
        "centroid": agent_centroid_m,
        "yaw": agent_yaw_rad,
        "extent": agent_extent_m,
        "velocity": agent_velocity_m
    }

class DecoratedAgentDataset(AgentDataset):
    def __init__(
        self,
        cfg: dict,
        zarr_dataset: ChunkedDataset,
        rasterizer: Rasterizer,
        perturbation: Optional[Perturbation] = None,
        agents_mask: Optional[np.ndarray] = None,
        min_frame_history: int = MIN_FRAME_HISTORY,
        min_frame_future: int = MIN_FRAME_FUTURE,
    ):
        assert perturbation is None, "AgentDataset does not support perturbation (yet)"
        super(DecoratedAgentDataset, self).__init__(cfg, zarr_dataset, rasterizer, perturbation, agents_mask,
                                                    min_frame_history, min_frame_future)
        render_context = RenderContext(
            raster_size_px=np.array(cfg["raster_params"]["raster_size"]),
            pixel_size_m=np.array(cfg["raster_params"]["pixel_size"]),
            center_in_raster_ratio=np.array(cfg["raster_params"]["ego_center"]),
        )
        self.sample_function = partial(
            decorated_generate_agent_sample,
            render_context=render_context,
            history_num_frames=cfg["model_params"]["history_num_frames"],
            history_step_size=cfg["model_params"]["history_step_size"],
            future_num_frames=cfg["model_params"]["future_num_frames"],
            future_step_size=cfg["model_params"]["future_step_size"],
            filter_agents_threshold=cfg["raster_params"]["filter_agents_threshold"],
            rasterizer=rasterizer
        )
        
    def get_frame(self, scene_index: int, state_index: int, track_id: Optional[int] = None) -> dict:
        frames = self.dataset.frames[get_frames_slice_from_scenes(self.dataset.scenes[scene_index])]

        tl_faces = self.dataset.tl_faces
        try:
            if self.cfg["raster_params"]["disable_traffic_light_faces"]:
                tl_faces = np.empty(0, dtype=self.dataset.tl_faces.dtype)  # completely disable traffic light faces
        except KeyError:
            warnings.warn(
                "disable_traffic_light_faces not found in config, this will raise an error in the future",
                RuntimeWarning,
                stacklevel=2,
            )
        data = self.sample_function(state_index, frames, self.dataset.agents, tl_faces, track_id)
        imgs = data["image"]
        k =cfg["model_params"]["history_num_frames"]
        target_positions = np.array(data["target_positions"], dtype=np.float32)
        target_velocity = np.array(data["target_velocity"], dtype=np.float32)
        target_yaws = np.array(data["target_yaws"], dtype=np.float32)
        return {
            "deep_layers" : imgs,
            "target_positions": target_positions,
            "target_velocity": target_velocity,
            "target_yaws" : target_yaws
        }

In [ ]:
def make_input_model():
    inputs = tfkl.Input(shape = (224,224,69))
    x = list()
    i = 0
    for j in range(6,67,6):
        t = tfkl.Conv2D(3, 7, padding="same")(tfkl.concatenate([inputs[...,i:j],inputs[...,-3:]]))
        t = tfkl.LeakyReLU(0.001)(t) 
        x.append(t) 
        i = j 
    t = tfkl.Conv2D(24, 5, padding="same")(tfkl.concatenate(x))
    t = tfkl.LeakyReLU(0.001)(t)
    t = tfkl.Conv2D(12, 3, padding="same")(t)
    t = tfkl.LeakyReLU(0.001)(t)
    t = tfkl.Conv2D(6, 3, padding="same")(t)
    t = tfkl.LeakyReLU(0.001)(t)
    t = tfkl.Conv2D(3, 3, padding="same")(t)
    curr_x = tfkl.LeakyReLU(0.001)(t)
    '''
    curr_x = x[0]
    for idx in range(1,len(x)):
        t = tfkl.Conv2D(3, 5, padding="same")(tfkl.concatenate([curr_x,x[idx]]))
        t = tfkl.LeakyReLU(0.001)(t)
        #t = tfkl.Conv2D(4, 3, padding="same")(t)
        #t = tfkl.LeakyReLU(0.001)(t) 
        curr_x = t #tfkl.Conv2D(3, 3, activation='relu', padding="same")(t) 
    '''
    model = tfk.Model(inputs,curr_x)
    return model

def make_cnn_model(nn):
    for i in range(1,len(nn.layers)):
        nn.layers[i].trainable = True
    inputs = tfkl.Input(shape = (raster_w,raster_h,input_deep))
    x = make_input_model()(inputs)
    x = nn(x)
    x = tfkl.Dropout(0.1)(x)
    x = tfkl.Dense(future_output_shape[0] * future_output_shape[1], kernel_regularizer=tfk.regularizers.l2(0.0001))(x)
    outputs = tfkl.Reshape(future_output_shape)(x)
    model = tfk.Model(inputs,outputs)
    return model


model = make_cnn_model(mob_net)
model.compile(optimizer="Adam", loss="mse", metrics=["mae"])
if load_weights:
    model.load_weights(weights_filepath)

In [ ]:
model.summary()

In [ ]:
train_cfg = cfg["train_data_loader"]
rasterizer = DecoratedSemBoxRasterizer(cfg, dm)
train_dataset = DecoratedAgentDataset(cfg, train_zarr, rasterizer, min_frame_history = 10, min_frame_future = 50)
Number_of_samples = len(train_dataset)
print(Number_of_samples)

In [ ]:
class TrainAgentGenerator():
    def __init__(self, agentdataset, number_of_samples):
        self.agentDataset = agentdataset
        self.number_of_samples = number_of_samples
    
    def __call__(self):
        for _ in range(self.number_of_samples * 10):
            i = np.random.randint(0,self.number_of_samples - 10)
            yield (self.agentDataset[i]["deep_layers"], self.agentDataset[i]["target_positions"])
            
train_gen = TrainAgentGenerator(train_dataset, Number_of_samples)
src_dataset = Dataset.from_generator(train_gen,(tf.float32, tf.float32))

In [ ]:
srcdata = src_dataset.batch(BATCHSIZE).prefetch(tf.data.experimental.AUTOTUNE)

In [ ]:
histv = model.fit(srcdata, epochs=1)

In [ ]:
#model.save_weights('./one_epoch_model.hdf5')

In [ ]:
#for d,t in srcdata:
#    y = model.evaluate(d,t,verbose = 1)